Question 1: Inverse kinematics of a Stanford manipulator

In [9]:
# Importing Libraries
import numpy as np

# Creating Class Robot and defining functions
class Robot():
  def __init__(self, endeffector_position, lengthsoflinks):
    self.endeffector_position = endeffector_position
    self.lengthsoflinks = lengthsoflinks
  
  def inverse_kinematics(self):
    theta1 = np.arctan(self.endeffector_position[1]/self.endeffector_position[0])
    r = np.sqrt(self.endeffector_position[0]**2 + self.endeffector_position[1]**2)
    s = self.endeffector_position[2] - self.lengthsoflinks[0]
    theta2 = np.arctan(s/r)
    d3 = np.sqrt(r**2 + s**2) - self.lengthsoflinks[1]
    print("First Solution: \n", "Theta1 = ", theta1, "\n Theta2 =", theta2,"\n Extension: ", d3, "\n")
    print("Second Solution: \n", "Theta1 = ", np.pi + theta1, "\n Theta2 =", np.pi - theta2,"\n Extension:", d3)


# Enter the coordinates of the end effector in the second input array. (x & y coordinates should be non zero to avoid singular configuration)
# Enter the values of d1 and a2 in the second input array.
Stanford = Robot([30,30,30],[10,10])
Stanford.inverse_kinematics()

First Solution: 
 Theta1 =  0.7853981633974483 
 Theta2 = 0.4405106630046985 
 Extension:  36.9041575982343 

Second Solution: 
 Theta1 =  3.9269908169872414 
 Theta2 = 2.7010819905850947 
 Extension: 36.9041575982343


Question 2: Inverse Kinematics of a SCARA Manipulator

In [13]:
# Importing Libraries
import numpy as np

# Creating Class Robot and defining functions
class Robot():
  def __init__(self, endeffector_position, lengthsoflinks):
    self.endeffector_position = endeffector_position
    self.lengthsoflinks = lengthsoflinks
  
  def inverse_kinematics(self):
    r = (self.endeffector_position[0]**2 + self.endeffector_position[1]**2 - self.lengthsoflinks[0]**2 - self.lengthsoflinks[1]**2)/(2 * self.lengthsoflinks[0] * self.lengthsoflinks[1])
    theta2 = np.arctan(np.sqrt(abs(1 - r**2)/(abs(r))))
    theta1 = np.arctan(self.endeffector_position[1]/self.endeffector_position[0]) - np.arctan((self.lengthsoflinks[1] * np.sin(theta2))/(self.lengthsoflinks[0] + self.lengthsoflinks[1] * np.cos(theta2)))
    d3 = self.endeffector_position[2]
    print("Solution: \n", "Theta1 = ", theta1, "\n Theta2 =", theta2,"\n Extension: ", d3, "\n")

# Enter the coordinates of the end effector in the second input array. (x & y coordinates should be non zero to avoid singular configuration)
# Enter the valus of a1, a2 in the second input array.
SCARA = Robot([68.3,68.3,-50],[50,50])
SCARA.inverse_kinematics()

Solution: 
 Theta1 =  0.5388252963736686 
 Theta2 = 0.49314573404755935 
 Extension:  -50 



Question 3: Calculate Joint velocities using end effector velocities

In [26]:
# Importing Libraries
import numpy as np
import matplotlib.pyplot as plt


# Creating Class Robot and defining functions
class Robot():
  def __init__(self, numberoflinks, DH_matrix,joint_types=-1):
    self.numberoflinks = numberoflinks
    self.DH_matrix = DH_matrix
    if(joint_types == -1):
      self.joint_types = [1]*numberoflinks
    else:
      self.joint_types = joint_types

  def transform_matrix(self):
    self.A = []
    self.R = []
    for i in range(self.numberoflinks):
      self.A.append(np.array([[np.cos(self.DH_matrix[i][0]), -np.sin(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), np.sin(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3]), self.DH_matrix[i][2] * np.cos(self.DH_matrix[i][0])],[np.sin(self.DH_matrix[i][0]), np.cos(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), -np.cos(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3]), self.DH_matrix[i][2] * np.sin(self.DH_matrix[i][0])],[0, np.sin(self.DH_matrix[i][3]), np.cos(self.DH_matrix[i][3]), self.DH_matrix[i][1]],[0, 0, 0, 1]])) 
      self.R.append(np.array([[np.cos(self.DH_matrix[i][0]), -np.sin(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), np.sin(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3])],[np.sin(self.DH_matrix[i][0]), np.cos(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), -np.cos(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3])],[0, np.sin(self.DH_matrix[i][3]), np.cos(self.DH_matrix[i][3])]]))
    self.A = np.array(self.A) # Array of Homogenous Transformation Matrices
    self.R = np.array(self.R) # Array of Rotation matrices
    
    # Creating array of R_0_n matrices (R01, R02, R03, R04 ...)
    self.R_0_n = []
    A = np.identity(3)
    for i in range(self.numberoflinks):
      self.R_0_n.append(A)
      A = np.matmul(A, self.R[i])
    self.R_0_n.append(A)
    self.R_0_n = np.array(self.R_0_n)

    # Creating array of H_0_n matrices (H01, H02, H03, H04 ...)
    self.H_0_n = []
    B = np.identity(4)
    for i in range(self.numberoflinks):
      self.H_0_n.append(B)
      B = np.matmul(B, self.A[i])
    self.H_0_n.append(B)
    self.H_0_n = np.array(self.H_0_n)
    
    # Calculating Final Transfornation Matrix
    T = np.asmatrix(np.identity(4))
    for j in range(self.numberoflinks):
        T = T * np.asmatrix(self.A[j]) #,dtype = 'int' for integer output
    self.T = T
    # print ("A Matrices are: \n", self.A)
    # print ("Final Transformation Matrix: \n",T)
    # print ("Rotation Matrices are: \n", self.R)
    # print ("Rotation Matrices w.r.t base frame are: \n", self.R_0_n)
  
  def manipulator_jacobian(self):
    # Creating z terms
    self.z = []
    k = np.array([[0],[0],[1]])
    for i in range(self.numberoflinks+1):
      self.z.append(np.matmul(self.R_0_n[i],k))
    self.z = np.array(self.z)

    # Creating O terms
    d = np.array([[0],[0],[0],[1]])
    self.O = np.matmul(self.H_0_n,d)[:self.numberoflinks+1]
    self.On = np.delete(self.O,(3),axis=1)

    # Creating Jacobian Manipulator (Here J1 will be 0th element of array J and so on)
    self.J = []
    for i in range(self.numberoflinks):
      if self.joint_types[i] == 1:
        X = self.On[-1] - self.On[i]
        J_v = np.cross(self.z[i], X,axis = 0)
        J_v = np.vstack((J_v, self.z[i]))
        self.J.append(J_v)
      else:
        J_v = np.vstack((self.z[i], np.array([[0],[0],[0]])))
        self.J.append(J_v)
    self.J = np.array(self.J)
    self.J_v = np.delete(self.J, (3,4,5), axis = 1)
    # print ("Manipulator Jacobian: \n", self.J)
    # print ("End effector position: \n", self.On[-1])
    # print ("End effector velocity: \n", "v = ", self.J_v, " x q_dot")

  def joint_velocities(self, xdot, ydot, zdot):
    X_dot  =np.array([[xdot],[ydot],[zdot]])
    J_v = self.J_v.reshape((3,3))
    Jacobian_Rank = np.linalg.matrix_rank(J_v)
    Augmentedmatrix_Rank = np.linalg.matrix_rank(np.hstack((J_v, X_dot)))
    if Jacobian_Rank != Augmentedmatrix_Rank:
      print("ERROR: Velocities provided are not possible for the given configuration")
      sys.exit()
    else:
      q_dot = np.linalg.pinv(J_v) @ X_dot
      print("Joint Velocities are given by: \n", q_dot)

# Initializing object "robot" and calculating result
# Mention the number of links in the first input
# Enter the DH table row wise in the order: [THETA, D, A, ALPHA]
# Enter Rho values for each link: 1 for Revolute and 0 for Prismatic
robot = Robot(3,[[np.pi/3, 0, 100, 0],[np.pi/6, 0, 100, 0],[np.pi/3, 0, 100, 0]],joint_types = [1,0,1])
robot.transform_matrix()
robot.manipulator_jacobian()
robot.joint_velocities(10,10,10)

Joint Velocities are given by: 
 [[-0.02679492]
 [-0.1       ]
 [10.        ]]


Question 6: Inverse Kinematics of a Spherical Wrist

In [30]:
# Importing Libraries
import numpy as np

# Creating Class Robot and defining functions
class Robot():
  def __init__(self, rotation_matrix):
    self.rotation_matrix = rotation_matrix
  
  def inverse_kinematics(self):
    self.rotation_matrix = np.reshape(self.rotation_matrix, (3,3))
    theta1 = np.arctan2(np.sqrt(abs(1 - self.rotation_matrix[2,2]**2)), self.rotation_matrix[2,2])
    theta2 = np.arctan2(-np.sqrt(abs(1 - self.rotation_matrix[2,2]**2)), self.rotation_matrix[2,2])
    phi1 = np.arctan2(self.rotation_matrix[1,2], self.rotation_matrix[0,2])
    phi2 = np.arctan2(-self.rotation_matrix[1,2], -self.rotation_matrix[0,2])
    psi1 = np.arctan2(-self.rotation_matrix[2,0], self.rotation_matrix[2,1])
    psi2 = np.arctan2(self.rotation_matrix[2,0], -self.rotation_matrix[2,1])
    print("First Solution: \n", "theta = ", theta1, "\n phi = ", phi1, "\n psi = ", psi1)
    print("Second Solution: \n", "theta = ", theta2, "\n phi = ", phi2, "\n psi = ", psi2)


# Enter the coordinates of the end effector in the second input array. (x & y coordinates should be non zero to avoid singular configuration)
# Enter the valus of a1, a2 in the second input array.
Spherical_Wrist = Robot([[2,0,0],[0,2,0],[0,0,2]])
Spherical_Wrist.inverse_kinematics()

First Solution: 
 theta =  0.7137243789447656 
 phi =  0.0 
 psi =  0.0
Second Solution: 
 theta =  -0.7137243789447656 
 phi =  0.0 
 psi =  0.0


Question 7: 3D Printer DH Parameters to calcualte Jacobian Matrix, Forward kinematics, End Effectors position and velocity.

In [4]:
# Importing Libraries
import numpy as np

# Creating Class Robot and defining functions
class Robot():
  def __init__(self, numberoflinks, DH_matrix,joint_types=-1):
    self.numberoflinks = numberoflinks
    self.DH_matrix = DH_matrix
    if(joint_types == -1):
      self.joint_types = [1]*numberoflinks
    else:
      self.joint_types = joint_types

  def forward_kinematics(self):
    self.A = []
    self.R = []
    for i in range(self.numberoflinks):
      self.A.append(np.array([[np.cos(self.DH_matrix[i][0]), -np.sin(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), np.sin(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3]), self.DH_matrix[i][2] * np.cos(self.DH_matrix[i][0])],[np.sin(self.DH_matrix[i][0]), np.cos(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), -np.cos(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3]), self.DH_matrix[i][2] * np.sin(self.DH_matrix[i][0])],[0, np.sin(self.DH_matrix[i][3]), np.cos(self.DH_matrix[i][3]), self.DH_matrix[i][1]],[0, 0, 0, 1]])) 
      self.R.append(np.array([[np.cos(self.DH_matrix[i][0]), -np.sin(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), np.sin(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3])],[np.sin(self.DH_matrix[i][0]), np.cos(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), -np.cos(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3])],[0, np.sin(self.DH_matrix[i][3]), np.cos(self.DH_matrix[i][3])]]))
    self.A = np.array(self.A) # Array of Homogenous Transformation Matrices
    self.R = np.array(self.R) # Array of Rotation matrices
    
    # Creating array of R_0_n matrices (R01, R02, R03, R04 ...)
    self.R_0_n = []
    A = np.identity(3)
    for i in range(self.numberoflinks):
      self.R_0_n.append(A)
      A = np.matmul(A, self.R[i])
    self.R_0_n.append(A)
    self.R_0_n = np.array(self.R_0_n)

    # Creating array of H_0_n matrices (H01, H02, H03, H04 ...)
    self.H_0_n = []
    B = np.identity(4)
    for i in range(self.numberoflinks):
      self.H_0_n.append(B)
      B = np.matmul(B, self.A[i])
    self.H_0_n.append(B)
    self.H_0_n = np.array(self.H_0_n)
    
    # Calculating Final Transfornation Matrix
    T = np.asmatrix(np.identity(4))
    for j in range(self.numberoflinks):
        T = T * np.asmatrix(self.A[j]) #,dtype = 'int' for integer output
    self.T = T
    # print ("A Matrices are: \n", self.A)
    print ("The forward kinematics equations in matrix form are: \n", "[p0 \n 1] = ", T ,"[p3 \n 1]")
    # print ("Rotation Matrices are: \n", self.R)
    # print ("Rotation Matrices w.r.t base frame are: \n", self.R_0_n)
  
  def manipulator_jacobian(self):
    # Creating z terms
    self.z = []
    k = np.array([[0],[0],[1]])
    for i in range(self.numberoflinks+1):
      self.z.append(np.matmul(self.R_0_n[i],k))
    self.z = np.array(self.z)

    # Creating O terms
    d = np.array([[0],[0],[0],[1]])
    self.O = np.matmul(self.H_0_n,d)[:self.numberoflinks+1]
    self.On = np.delete(self.O,(3),axis=1)

    # Creating Jacobian Manipulator (Here J1 will be 0th element of array J and so on)
    self.J = []
    for i in range(self.numberoflinks):
      if self.joint_types[i] == 1:
        X = self.On[-1] - self.On[i]
        J_v = np.cross(self.z[i], X,axis = 0)
        J_v = np.vstack((J_v, self.z[i]))
        self.J.append(J_v)
      else:
        J_v = np.vstack((self.z[i], np.array([[0],[0],[0]])))
        self.J.append(J_v)
    self.J = np.array(self.J)
    self.J_v = np.delete(self.J, (3,4,5), axis = 1)
    print ("Manipulator Jacobian: \n", self.J)
    print ("End effector position: \n", self.On[-1])
    print ("End effector velocity: \n", "v = ", self.J_v, " x q_dot")

# Initializing object "robot" and calculating result
# Mention the number of links in the first input
# Enter the DH table row wise in the order: [THETA, D, A, ALPHA]
# Enter Rho values for each link: 1 for Revolute and 0 for Prismatic
DPrinter = Robot(3,[[0, 50 + 10, 0, -np.pi/2],[-np.pi/2, 50 + 10, 0, -np.pi/2],[0, 50 + 10, 0, 0]],joint_types = [0,0,0])
DPrinter.forward_kinematics()
DPrinter.manipulator_jacobian()

The forward kinematics equations in matrix form are: 
 [p0 
 1] =  [[ 6.123234e-17  6.123234e-17  1.000000e+00  6.000000e+01]
 [-6.123234e-17 -1.000000e+00  6.123234e-17  6.000000e+01]
 [ 1.000000e+00 -6.123234e-17 -6.123234e-17  6.000000e+01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]] [p3 
 1]
Manipulator Jacobian: 
 [[[ 0.000000e+00]
  [ 0.000000e+00]
  [ 1.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]]

 [[ 0.000000e+00]
  [ 1.000000e+00]
  [ 6.123234e-17]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]]

 [[ 1.000000e+00]
  [ 6.123234e-17]
  [-6.123234e-17]
  [ 0.000000e+00]
  [ 0.000000e+00]
  [ 0.000000e+00]]]
End effector position: 
 [[60.]
 [60.]
 [60.]]
End effector velocity: 
 v =  [[[ 0.000000e+00]
  [ 0.000000e+00]
  [ 1.000000e+00]]

 [[ 0.000000e+00]
  [ 1.000000e+00]
  [ 6.123234e-17]]

 [[ 1.000000e+00]
  [ 6.123234e-17]
  [-6.123234e-17]]]  x q_dot
